# 対話管理（有限オートマトンによる方法）
ここでは、対話管理を実装します。まずは有限オートマトンによる方法で行います。


## 事前の設定
- 特になし

In [1]:
# 必要なラブラリを読み込む

import sys, os

有限オートマトンを定義します。ここでは状態と遷移を別々に定義します。

ここでは、下記の有限オートマトンを実装してみます。ただし、検索条件が揃う（「検索します。」というシステム発話）までを扱います。

状態番号は下記とします。
- 左の上から４つが上から順に０～３
- 右の上から３つが上から順に４～６

<img src="./img/automaton.png" style="width: 800px;"/>

## 状態の定義

まずは、状態の定義です。各状態はシステムの発話に対応します。

In [2]:
# 状態の定義

# 状態番号、対応するシステム発話
states = [
    [0, 'こんにちは。京都レストラン案内です。どの地域のレストランをお探しですか。'],
    [1, 'どのような料理がお好みですか。'],
    [2, 'ご予算はおいくらぐらいですか。'],
    [3, '検索します。'],
    [4, '地域名を「京都駅近辺のようにおっしゃってください。'],
    [5, '和食・洋食・中華・ファストフードからお選びください。'],
    [6, '予算を「3000円以下」のようにおっしゃってください。']    
]

start_state = 0
end_state = 3


## 遷移の定義
次に、遷移を定義します。遷移元と遷移先の状態番号、条件となるユーザ発話の情報で構成します。ユーザ発話の情報は、ここでは言語理解で設計したスロット名（place や genre など）にします。また、後で実装しますが、スロット値は変数として保存しておきます。また、遷移条件の「None」はそれより上の条件にマッチしなかった場合の「それ以外の入力」に相当します。

In [3]:
# 遷移の定義

# 遷移元状態番号、遷移先状態番号、遷移条件（スロット名）
transitions = [
    [0, 1, 'place'],
    [0, 4, None],
    [1, 2, 'genre'],
    [1, 5, None],
    [2, 3, 'budget'],
    [2, 6, None],
    [4, 1, 'place'],
    [4, 4, None],
    [5, 2, 'genre'],
    [5, 5, None],
    [6, 3, 'budget'],
    [6, 6, None]
]

## 制御部の実装
では、定義した状態と遷移に基づいて対話を制御します。内部変数として、現在の状態番号を保持し、入力であるユーザ発話に応じてシステム発話を出力し、状態を遷移させます。

In [4]:
# 現在の内部状態を初期状態にする
current_state = start_state

# 遷移条件にマッチしたユーザ発話を保持する
context_user_utterance = []

# 入力であるユーザ発話に応じてシステム発話を出力し、内部状態を遷移させる
# ただし、ユーザ発話の情報は「意図、スロット名、スロット値」のlistとする
def enter(user_utterance):
    
    global current_state
    
    # フレーム名に対して行う
    # 最初の0番目のindexは1発話に対して複数のスロットが抽出された場合に対応するため
    # ここでは1発話につき１つのフレームしか含まれないという前提
    input_frame_name = user_utterance[0]['slot_name']
    input_frame_value = user_utterance[0]['slot_value']
    
    system_utterance = ""
    
    # 現在の状態からの遷移に対して入力がマッチするか検索
    for trans in transitions:
        
        # 条件の遷移元が現在の状態か
        if trans[0] == current_state:
            
            # 無条件に遷移
            if trans[2] is None:
                current_state = trans[1]
                system_utterance = get_system_utterance()
                break
            
            # 条件にマッチすれば遷移
            if trans[2] == input_frame_name:
                context_user_utterance.append([input_frame_name, input_frame_value])
                current_state = trans[1]
                system_utterance = get_system_utterance()
                break
    
    # 修了状態に達したら初期状態へ戻す
    if current_state == end_state:
        current_state = end_state
    
    return system_utterance

# 初期状態にリセットする
def reset():
    global current_state
    current_state = start_state

# 指定された状態に対応するシステムの発話を取得
def get_system_utterance():
    
    global current_state
    
    utterance = ""
    
    for state_ in states:
        if current_state == state_[0]:
            utterance = state_[1]
    
    return utterance

## 対話管理のテスト
実装した対話管理をテストしてみます。

In [5]:
reset()

# 初期状態の発話を表示
print("システム発話 : " + get_system_utterance())

システム発話 : こんにちは。京都レストラン案内です。どの地域のレストランをお探しですか。


In [6]:
# ユーザ発話を設定
user_utterance = [{'slot_name': 'place', 'slot_value': '京都駅周辺'}]
print('ユーザ発話')
print(user_utterance)

print()

# 次のシステム発話を表示
print('システム発話')
print(enter(user_utterance))

ユーザ発話
[{'slot_name': 'place', 'slot_value': '京都駅周辺'}]

システム発話
どのような料理がお好みですか。


In [7]:
# 誤った発話を入力してみる

# ユーザ発話を設定
user_utterance = [{'slot_name': 'place', 'slot_value': '新宿'}]
print('ユーザ発話')
print(user_utterance)
print()

# 次のシステム発話を表示
print('システム発話')
print(enter(user_utterance))

ユーザ発話
[{'slot_name': 'place', 'slot_value': '新宿'}]

システム発話
和食・洋食・中華・ファストフードからお選びください。


In [8]:
# ユーザ発話を設定
user_utterance = [{'slot_name': 'genre', 'slot_value': '和食'}]
print('ユーザ発話')
print(user_utterance)
print()

# 次のシステム発話を表示
print('システム発話')
print(enter(user_utterance))

ユーザ発話
[{'slot_name': 'genre', 'slot_value': '和食'}]

システム発話
ご予算はおいくらぐらいですか。


In [9]:
# ユーザ発話を設定
user_utterance = [{'slot_name': 'budget', 'slot_value': '3000円以下'}]
print('ユーザ発話')
print(user_utterance)
print()

# 次のシステム発話を表示
print('システム発話')
print(enter(user_utterance))

ユーザ発話
[{'slot_name': 'budget', 'slot_value': '3000円以下'}]

システム発話
検索します。
